# Automated KPI email alerts for Business Users

## KPIs

- Total Number of Orders
- Gross Revenue
- Discount
- Net Revenue
- Average Delivery Time of Orders
- Average Preparation Time of Orders
- Total Users
- New Users in the time period
- User Retention in the time period


### - Percentage Change over the time period

## Data Sources

- Database - Mysql
- Cloud - Google Cloud Storage
- Data Access - BigQuery MySQL

## Alerting Module

- Python - email, Request, and others
- Gmail alert

## BACKGROUND

### The Pulse of Progress: Automating KPI Alerts for a Data-Driven Business

In today's dynamic business landscape, staying ahead of the curve isn't just about tracking numbers; it's about understanding their implications in real-time and acting decisively. Simply monitoring dashboards isn't enough. The true value lies in proactive insights, delivered precisely when they matter most. This is where the magic of automated KPI alerts comes in.

Key Performace Indicators (KPI) are the vital signs of a business's operational health, reflecting it's ability to navigate market fluctuations and achieve strategic objectives. Automated KPI alerts act as early warning systems, signaling potential headwinds or tailwinds as they begin to materialize.

Automated alerts empower us to move beyond reactive analysis to proactive risk management and opportunity identification. They help us connect micro-level operational data with macro-level economic trends, leading to more informed and timely strategic decisions

Some Key points are :

The specific KPI and its current value.
The magnitude and direction of the change (e.g., a 15% drop in weekly sales).
Comparison to previous periods (daily, weekly, monthly averages).
Potential contributing factors (if integrated with other data sources).
Links to relevant dashboards or reports for deeper investigation.


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta
import datetime

import os
import pickle
import base64

# Gmail API utils
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from googleapiclient.discovery import build
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

# Write excel files with KPIs

from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
project_id = "mixing-testnet"
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

In [4]:
def errors(error,error_name,code,platform):
    
    """Write the error into an error file"""
    
    today = pd.to_datetime(date.today())
    current_time = today.strftime("%H:%M:%S")
    today=today.strftime('%Y-%m-%d')
    error=str(error)
    df=pd.DataFrame([[code,platform,error,error_name,today,current_time]],columns=['Code','Platform','Error','Error_Name','run_date','run_time'])
    #target_table = "mixing-testnet.10_min_Delivery_error.Error_logs_shipsy_order"
    target_table = "10_min_Delivery_error.Error_logs_daily_alert"
    #credential_file = ("mixing-testnet-4df1b518f6ef.json")
    project_id = "mixing-testnet"
    #credential = Credentials.from_service_account_file(credential_file)
    # Location for BQ job, it needs to match with destination table location
    # Save Pandas dataframe to BQ
#     df['run_date']=df['run_date'].astype(str)
#    df.to_gbq(target_table,project_id=project_id,progress_bar=True,if_exists='replace', table_schema=None,credentials=credential)
    df.to_gbq(target_table, project_id=project_id, progress_bar=True,if_exists='append', table_schema=None)

In [5]:
# Data extraction from Mysql, BigQuery

try:
  query_send = '''
  SELECT o.Order_date, o.Order_time, o.Order_id, o.Outlet_name, o.Order_status, o.Order_source, o.Total_discount, o.Total_tax, o.Subtotal, o.Total_charge, 
  o.Merchant_offer_code, o.Customer_mobile, o.Customer_name, o.Loyalty_amount, o.Customer_email, o.Revenue, os.Id, os.State_current, os.Prepration_time,
  d.Rider_Name, d.Single_Run, d.Order_Time, d.Reassigned_Rider, 
  d.order_to_makeline, d.makeline_to_order_assign, d.Order_Assign_to_bike, d.store_to_gate_time,
  d.gate_to_delivery, d.delivery_to_store, d.makeline_Delivery_final, d.slot_time, d.overall_delivery_category
  FROM `mixing-testnet.10min_Orderwise.stg_10min_orders` o 
  INNER JOIN `mixing-testnet.10min_Orderwise.stg_10min_order_status` os 
  ON (o.Order_id = os.Id)
  INNER JOIN `mixing-testnet.10min_Delivery.stg_10min_delivery_reports` d 
  ON (o.Order_id = d.Order_Id)
  WHERE o.Merchant_offer_code != 'SACHIN' AND o.Order_status = 'DELIVERED' '''
  df_merged = client.query(query_send).to_dataframe()
except Exception as e:
  errors(e, 'big_query_error', 'Daily_Alert', 'voosh')

In [6]:
#initialise data

#Set Flags

Data_Exists_Daily = False
Data_Exists_Weekly = False
Data_Exists_Monthly = False

#Week Month Flags
Is_Month_Start = False
Is_Week_Start = False

In [7]:
# Check if week start / month start

T_Date = pd.to_datetime(date.today())
T_1_Date =  pd.to_datetime(date.today() - timedelta(days = 1))
T_2_Date = pd.to_datetime(date.today() - timedelta(days = 2))
if (T_Date.is_month_start)|(T_1_Date.is_month_start)|(T_2_Date.is_month_start):
  Is_Month_Start = True
  month_1 = T_Date.month -1
  month_2 = T_Date.month -2 
if (T_Date.day_name()=='Monday'):
  Is_Week_Start = True 
  week_1 = T_Date.week -1
  week_2 = T_Date.week -2 

T_Date,T_1_Date,T_2_Date

(Timestamp('2022-05-30 00:00:00'),
 Timestamp('2022-05-29 00:00:00'),
 Timestamp('2022-05-28 00:00:00'))

In [8]:
#Initial Data Prep

def convert_to_minutes(x): #11:33:23 #convert_to_minutes('00:02:30')
  return round((float(x.split(':')[0])*60) + (float(x.split(':')[1])*1) + (float(x.split(':')[2])/60),2)

try:
  df_merged['Prep_Time'] = df_merged['Prepration_time'].fillna('00:00:00').apply(convert_to_minutes)
  df_merged['MFR_Marked'] = (df_merged['Prep_Time']==0) 
  df_merged['Year'] = pd.to_datetime(df_merged['Order_date']).dt.year
  df_merged['Month'] = pd.to_datetime(df_merged['Order_date']).dt.month
  df_merged['Week'] = pd.to_datetime(df_merged['Order_date']).dt.isocalendar().week
#store_name = df_merged['Outlet_name'].unique()
except Exception as e:
  errors(e, 'Initial_Data_Prep', 'Daily_Alert', 'voosh')

In [9]:
#Daily Values

try:
  T_1 = (date.today() - timedelta(days = 1)).strftime("%Y-%m-%d") #yesterday 
  T_2 = (date.today() - timedelta(days = 2)).strftime("%Y-%m-%d") #day before yesterday
  #print(T_1,T_2)
  df_T_1 = df_merged[df_merged['Order_date']==T_1]
  df_T_2 = df_merged[df_merged['Order_date']==T_2]
  print(len(df_T_1),len(df_T_2))
  #daily_alert =pd.DataFrame(columns=['(T-2)','Yesterday(T-1)'],index=['Total Orders','Gross Revenue $', 
  daily_alert =pd.DataFrame(columns=[T_2,T_1],index=['Total Orders','Gross Revenue $', 
                                                                                          'Discount $','Net Revenue $', 'Avg Delivery Time',
                                                                                          'Avg Prep time', 'MFR%','New Users', 'Old Users'])
  if len(df_T_1) !=0 and len(df_T_2) !=0:
    #Prepare T-1 Alert
    #print('here')
    daily_alert.loc['Total Orders',T_1] = len(df_T_1)
    daily_alert.loc['Gross Revenue $',T_1] = df_T_1['Revenue'].sum() + df_T_1['Total_discount'].sum()
    daily_alert.loc['Discount $',T_1] = df_T_1['Total_discount'].sum()
    daily_alert.loc['Net Revenue $',T_1] = df_T_1['Revenue'].sum()
    daily_alert.loc['Avg Delivery Time',T_1] = round(df_T_1[df_T_1['makeline_Delivery_final']<=100]['makeline_Delivery_final'].mean(),2)
    daily_alert.loc['Avg Prep time',T_1] = round(df_T_1[df_T_1['Prep_Time']!=0]['Prep_Time'].mean(),2)
    daily_alert.loc['MFR%',T_1] = round((len(df_T_1[~df_T_1['MFR_Marked']])/(len(df_T_1)))*100,2)
    daily_alert.loc['Total Users',T_1] = len(set(df_T_1['Customer_mobile']))
    daily_alert.loc['New Users',T_1] = len(set(df_T_1['Customer_mobile'])) - len(set(df_merged[df_merged['Order_date']!=T_1]['Customer_mobile']).intersection(set(df_T_1['Customer_mobile'])))
    daily_alert.loc['Old Users',T_1] = len(set(df_merged[df_merged['Order_date']!=T_1]['Customer_mobile']).intersection(set(df_T_1['Customer_mobile'])))

    #Prepare T-2 Alert
    daily_alert.loc['Total Orders',T_2] = len(df_T_2)
    daily_alert.loc['Gross Revenue $',T_2] = df_T_2['Revenue'].sum() + df_T_2['Total_discount'].sum()
    daily_alert.loc['Discount $',T_2] = df_T_2['Total_discount'].sum()
    daily_alert.loc['Net Revenue $',T_2] = df_T_2['Revenue'].sum()
    daily_alert.loc['Avg Delivery Time',T_2] = round(df_T_2[df_T_2['makeline_Delivery_final']<=100]['makeline_Delivery_final'].mean(),2)
    daily_alert.loc['Avg Prep time',T_2] = round(df_T_2[df_T_2['Prep_Time']!=0]['Prep_Time'].mean(),2)
    daily_alert.loc['MFR%',T_2] = round((len(df_T_2[~df_T_2['MFR_Marked']])/(len(df_T_2)))*100,2)
    daily_alert.loc['Total Users',T_2] = len(set(df_T_2['Customer_mobile']))
    daily_alert.loc['New Users',T_2] = len(set(df_T_2['Customer_mobile'])) - len(set(df_merged[(df_merged['Order_date']!=T_1) & (df_merged['Order_date']!=T_2)]['Customer_mobile']).intersection(set(df_T_2['Customer_mobile'])))
    daily_alert.loc['Old Users',T_2] = len(set(df_merged[(df_merged['Order_date']!=T_1) & (df_merged['Order_date']!=T_2)]['Customer_mobile']).intersection(set(df_T_2['Customer_mobile'])))

    daily_alert['%chg'] = daily_alert.pct_change(axis=1)[T_1].round(2)
    Data_Exists_Daily = True
  else:
    Data_Exists_Daily = False
    errors(e, 'Data_Doesnt_Exist_For_Daily_Comparison', 'Daily_Alert', 'voosh')
except Exception as e:
  errors(e, 'Daily_Values', 'Daily_Alert', 'voosh')


40 23


In [11]:
#daily_alert

,2022-05-28,2022-05-29,%chg,Week 20,Week 21,%WChg
Total Orders,23,40,0.74,240.00,245.00,0.02
Gross Revenue $,3307.58,5941.4,0.80,46508.69,40075.09,-0.14
Discount $,1232.58,1813.4,0.47,11682.69,12598.09,0.08
Net Revenue $,2075,4128,0.99,34826.00,27477.00,-0.21
Avg Delivery Time,15.63,12.36,-0.21,15.15,14.01,-0.08
Avg Prep time,2.5,2.76,0.10,3.99,3.25,-0.19
MFR%,100.0,97.5,-0.03,96.67,94.69,-0.02
New Users,10,21,1.10,107.00,106.00,-0.01
Old Users,12,16,0.33,62.00,68.00,0.10
Total Users,22,37,0.68,169.00,174.00,0.03


## CREATE WEEKLY MONTHLY AND DAILY ALERTS

In [ ]:
#Is_Week_Start

In [10]:
#if Is_Week_Start:
#week_1 = T_Date.week -1 # to be commented, test purpose
#week_2 = T_Date.week -2 # to be commented, test purpose
#if 1==1: # to be commented, test purpose
try:

  if Is_Week_Start:
  #  daily_alert[['Last Week W-1','W-2']] = 0.0
    df_W_1 = df_merged[df_merged['Week']==week_1]
    df_W_2 = df_merged[df_merged['Week']==week_2]
  #  print(len(df_W_1),len(df_W_2))

    if (len(df_W_1) !=0 and len(df_W_2) !=0):

      #Prepare T-2 Alert
      daily_alert.loc['Total Orders',('Week ' + str(week_2))] = round(len(df_W_2),0)
      daily_alert.loc['Gross Revenue $',('Week ' + str(week_2))] = df_W_2['Revenue'].sum() + df_W_2['Total_discount'].sum()
      daily_alert.loc['Discount $',('Week ' + str(week_2))] = df_W_2['Total_discount'].sum()
      daily_alert.loc['Net Revenue $',('Week ' + str(week_2))] = df_W_2['Revenue'].sum()
      daily_alert.loc['Avg Delivery Time',('Week ' + str(week_2))] = round(df_W_2[df_W_2['makeline_Delivery_final']<=100]['makeline_Delivery_final'].mean(),2)
      daily_alert.loc['Avg Prep time',('Week ' + str(week_2))] = round(df_W_2[df_W_2['Prep_Time']!=0]['Prep_Time'].mean(),2)
      daily_alert.loc['MFR%',('Week ' + str(week_2))] = round((len(df_W_2[~df_W_2['MFR_Marked']])/len(df_W_2))*100,2)
      daily_alert.loc['Total Users',('Week ' + str(week_2))] = int(round(len(set(df_W_2['Customer_mobile'])),0))
      daily_alert.loc['New Users',('Week ' + str(week_2))] = int(round(len(set(df_W_2['Customer_mobile'])) - len(set(df_merged[(df_merged['Week']!=week_1) & (df_merged['Week']!=week_2)]['Customer_mobile']).intersection(set(df_W_2['Customer_mobile']))),0))
      daily_alert.loc['Old Users',('Week ' + str(week_2))] = int(round(len(set(df_merged[(df_merged['Week']!=week_1) & (df_merged['Week']!=week_2)]['Customer_mobile']).intersection(set(df_W_2['Customer_mobile']))),0))

      #Prepare W-1 Alert
      daily_alert.loc['Total Orders',('Week ' + str(week_1))] = round(len(df_W_1),0)
      daily_alert.loc['Gross Revenue $',('Week ' + str(week_1))] = df_W_1['Revenue'].sum() + df_W_1['Total_discount'].sum()
      daily_alert.loc['Discount $',('Week ' + str(week_1))] = df_W_1['Total_discount'].sum()
      daily_alert.loc['Net Revenue $',('Week ' + str(week_1))] = df_W_1['Revenue'].sum()
      daily_alert.loc['Avg Delivery Time',('Week ' + str(week_1))] = round(df_W_1[df_W_1['makeline_Delivery_final']<=100]['makeline_Delivery_final'].mean(),2)
      daily_alert.loc['Avg Prep time',('Week ' + str(week_1))] = round(df_W_1[df_W_1['Prep_Time']!=0]['Prep_Time'].mean(),2)
      daily_alert.loc['MFR%',('Week ' + str(week_1))] = round((len(df_W_1[~df_W_1['MFR_Marked']])/len(df_W_1))*100,2)
      daily_alert.loc['Total Users',('Week ' + str(week_1))] = len(set(df_W_1['Customer_mobile']))
      daily_alert.loc['New Users',('Week ' + str(week_1))] = len(set(df_W_1['Customer_mobile'])) - len(set(df_merged[df_merged['Week']!=week_1]['Customer_mobile']).intersection(set(df_W_1['Customer_mobile'])))
      daily_alert.loc['Old Users',('Week ' + str(week_1))] = len(set(df_merged[df_merged['Week']!=week_1]['Customer_mobile']).intersection(set(df_W_1['Customer_mobile'])))

      daily_alert['%WChg'] = daily_alert[[('Week ' + str(week_2)),('Week ' + str(week_1))]].pct_change(axis=1)[('Week ' + str(week_1))].round(2).fillna(0)
      Data_Exists_Weekly = True

    else:
      Data_Exists_Weekly = False
      errors(e, 'Data_Doesnt_Exist_For_Weekly_Comparison', 'Daily_Alert', 'voosh')
except Exception as e:
  errors(e, 'Weekly_Values', 'Daily_Alert', 'voosh')



In [ ]:
#Monthly Alert

#month_1=5 # to be commented
#month_2=4 # to be commented
#if 1==1: # to be commented
try:

  if Is_Month_Start:

  #  daily_alert[['Last Week W-1','W-2']] = 0.0
    df_M_1 = df_merged[df_merged['Month']==month_1]
    df_M_2 = df_merged[df_merged['Month']==month_2]
  #  print(len(df_M_1),len(df_M_2))

    if (len(df_M_1) !=0 and len(df_M_2) !=0):
      #Prepare M-2 Alert
      daily_alert.loc['Total Orders',('Month ' + str(month_2))] = len(df_M_2)
      daily_alert.loc['Gross Revenue $',('Month ' + str(month_2))] = df_M_2['Revenue'].sum() + df_M_2['Total_discount'].sum()
      daily_alert.loc['Discount $',('Month ' + str(month_2))] = df_M_2['Total_discount'].sum()
      daily_alert.loc['Net Revenue $',('Month ' + str(month_2))] = df_M_2['Revenue'].sum()
      daily_alert.loc['Avg Delivery Time',('Month ' + str(month_2))] = round(df_M_2[df_M_2['makeline_Delivery_final']<=100]['makeline_Delivery_final'].mean(),2)
      daily_alert.loc['Avg Prep time',('Month ' + str(month_2))] = round(df_M_2[df_M_2['Prep_Time']!=0]['Prep_Time'].mean(),2)
      daily_alert.loc['MFR%',('Month ' + str(month_2))] = round((len(df_M_2[~df_M_2['MFR_Marked']])/len(df_M_2)*100),2)
      daily_alert.loc['Total Users',('Month ' + str(month_2))] = len(set(df_M_2['Customer_mobile']))
      daily_alert.loc['New Users',('Month ' + str(month_2))] = int(round(len(set(df_M_2['Customer_mobile'])) - len(set(df_merged[(df_merged['Month']!=month_1) & (df_merged['Month']!=month_2)]['Customer_mobile']).intersection(set(df_M_2['Customer_mobile']))),0))
      daily_alert.loc['Old Users',('Month ' + str(month_2))] = int(round(len(set(df_merged[(df_merged['Month']!=month_1) & (df_merged['Month']!=month_2)]['Customer_mobile']).intersection(set(df_M_2['Customer_mobile']))),0))

      #Prepare M-1 Alert
      daily_alert.loc['Total Orders',('Month ' + str(month_1))] = len(df_M_1)
      daily_alert.loc['Gross Revenue $',('Month ' + str(month_1))] = df_M_1['Revenue'].sum() + df_M_1['Total_discount'].sum()
      daily_alert.loc['Discount $',('Month ' + str(month_1))] = df_M_1['Total_discount'].sum()
      daily_alert.loc['Net Revenue $',('Month ' + str(month_1))] = df_M_1['Revenue'].sum()
      daily_alert.loc['Avg Delivery Time',('Month ' + str(month_1))] = round(df_M_1[df_M_1['makeline_Delivery_final']<=100]['makeline_Delivery_final'].mean(),2)
      daily_alert.loc['Avg Prep time',('Month ' + str(month_1))] = round(df_M_1[df_M_1['Prep_Time']!=0]['Prep_Time'].mean(),2)
      daily_alert.loc['MFR%',('Month ' + str(month_1))] = round((len(df_M_1[~df_M_1['MFR_Marked']])/len(df_M_1))*100,2)
      daily_alert.loc['Total Users',('Month ' + str(month_1))] = len(set(df_M_1['Customer_mobile']))
      daily_alert.loc['New Users',('Month ' + str(month_1))] = len(set(df_M_1['Customer_mobile'])) - len(set(df_merged[df_merged['Month']!=month_1]['Customer_mobile']).intersection(set(df_M_1['Customer_mobile'])))
      daily_alert.loc['Old Users',('Month ' + str(month_1))] = len(set(df_merged[df_merged['Month']!=month_1]['Customer_mobile']).intersection(set(df_M_1['Customer_mobile'])))

      daily_alert['%MChg'] = daily_alert[[('Month ' + str(month_2)),('Month ' + str(month_1))]].pct_change(axis=1)[('Month ' + str(month_1))].round(2)
      Data_Exists_Monthly = True

    else:
      Data_Exists_Monthly = False
      errors(e, 'Data_Doesnt_Exist_For_Monthly_Comparison', 'Daily_Alert', 'voosh')
except Exception as e:
  errors(e, 'Monthly_Values', 'Daily_Alert', 'voosh')



In [ ]:
#daily_alert

In [ ]:
# Change the decimals on Total Orders, New Users, Old Users, Total Users to 0
try:

  columns_change=[]
  rows_change = ['Total Orders','New Users','Old Users','Total Users']
  if Data_Exists_Daily: 
    columns_change.append(T_1)
    columns_change.append(T_2)
  if Data_Exists_Weekly:
    columns_change.append('Week ' + str(week_1))
    columns_change.append('Week ' + str(week_2)) 
  if Data_Exists_Monthly:
    columns_change.append('Month '+str(month_1))
    columns_change.append('Month ' +str(month_2)) 

  for row in rows_change:
    for column in columns_change:
      if pd.notna(daily_alert.loc[row,column]):
        daily_alert.loc[row,column] = str(int(daily_alert.loc[row,column]))
  
  # Change Nans in the df into '-
  daily_alert.fillna('-',inplace=True)

except Exception as e:
  errors(e, 'Format_Change', 'Daily_Alert', 'voosh')


## CREATE WEEKLY MONTHLY AND DAILY CSV FILES

In [ ]:
#orders,gross,discount,revenue,avg_delivery,avg_prep,new_users,old_users = []
# Daily CSV
try:

  if Data_Exists_Daily:

    store_name = df_T_1['Outlet_name'].unique()
    df_daily_csv_T1 =pd.DataFrame(columns=['Date','Total Orders','Gross Revenue $', 'Discount $','Net Revenue $', 'Avg Delivery Time',
                                        'Avg Prep time', 'MFR%','New Users', 'Old Users', 'Total Users'], index=[store_name])
    df_daily_csv_T2 =pd.DataFrame(columns=['Date','Total Orders','Gross Revenue $', 'Discount $','Net Revenue $', 'Avg Delivery Time',
                                        'Avg Prep time', 'MFR%', 'New Users', 'Old Users','Total Users'], index=[store_name])

    for i in store_name:
      df = df_T_1[df_T_1['Outlet_name']==i]
      df_daily_csv_T1.loc[i,'Date'] = T_1
      df_daily_csv_T1.loc[i,'Total Orders'] = df['Order_id'].count()
      df_daily_csv_T1.loc[i,'Net Revenue $'] = df['Revenue'].sum()
      df_daily_csv_T1.loc[i,'Discount $'] = df['Total_discount'].sum()
      df_daily_csv_T1.loc[i,'Gross Revenue $'] = float(df['Revenue'].sum() + df['Total_discount'].sum())

      df_daily_csv_T1.loc[i,'Avg Delivery Time'] = round(df[df['makeline_Delivery_final']<=100]['makeline_Delivery_final'].mean(),2)
      if (len(df) and len(df[~df['MFR_Marked']]) > 0):
        df_daily_csv_T1.loc[i,'MFR%'] = round((len(df[~df['MFR_Marked']])/len(df))*100,2)
      else:
        df_daily_csv_T1.loc[i,'MFR%'] = 0
      df_daily_csv_T1.loc[i,'Avg Prep time'] = round(df[df['Prep_Time']!=0]['Prep_Time'].mean(),2)
      df_daily_csv_T1.loc[i,'New Users'] = len(set(df['Customer_mobile'])) - len(set(df_merged[df_merged['Order_date']!=T_1]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
      df_daily_csv_T1.loc[i,'Old Users'] = len(set(df_merged[df_merged['Order_date']!=T_1]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
      df_daily_csv_T1.loc[i,'Total Users'] = len(set(df['Customer_mobile']))

    for i in store_name:
      df = df_T_2[df_T_2['Outlet_name']==i]
      df_daily_csv_T2.loc[i,'Date'] = T_2
      df_daily_csv_T2.loc[i,'Total Orders'] = df['Order_id'].count()
      df_daily_csv_T2.loc[i,'Net Revenue $'] = df['Revenue'].sum()
      df_daily_csv_T2.loc[i,'Discount $'] = df['Total_discount'].sum()
      df_daily_csv_T2.loc[i,'Gross Revenue $'] = float(df['Revenue'].sum() + df['Total_discount'].sum())

      df_daily_csv_T2.loc[i,'Avg Delivery Time'] = round(df[df['makeline_Delivery_final']<=100]['makeline_Delivery_final'].mean(),2)
      df_daily_csv_T2.loc[i,'Avg Prep time'] = round(df[df['Prep_Time']!=0]['Prep_Time'].mean(),2)
      if (len(df) and len(df[~df['MFR_Marked']]) > 0):
        df_daily_csv_T2.loc[i,'MFR%'] = round((len(df[~df['MFR_Marked']])/len(df)*100),2)
      else:
        df_daily_csv_T2.loc[i,'MFR%'] = 0
      df_daily_csv_T2.loc[i,'Total Users'] = len(set(df['Customer_mobile']))
    
      df_daily_csv_T2.loc[i,'New Users'] = len(set(df['Customer_mobile'])) - len(set(df_merged[(df_merged['Order_date']!=T_1) & (df_merged['Order_date']!=T_2)]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
      df_daily_csv_T2.loc[i,'Old Users'] = len(set(df_merged[(df_merged['Order_date']!=T_1) & (df_merged['Order_date']!=T_2)]['Customer_mobile']).intersection(set(df['Customer_mobile'])))

    #  daily_alert.loc['New Users','(T-2)'] = len(set(df_merged[df_merged['Order_date']!=T_1]['Customer_mobile']).intersection(set(df_T_1['Customer_mobile'])))
    #  daily_alert.loc['Old Users','(T-2)'] = len(set(df_T_2['Customer_mobile']).difference(set(df_merged[(df_merged['Order_date']!=T_1)&(df_merged['Order_date']!=T_2)]['Customer_mobile'])))
    df_daily_csv = pd.concat([df_daily_csv_T1,df_daily_csv_T2])
    df_daily_csv.reset_index (inplace=True)
    df_daily_csv.rename(columns = {'level_0':'Store Name'}, inplace = True)
    df_daily_csv=df_daily_csv.sort_values(by=['Store Name'])

except Exception as e:
  errors(e, 'Daily_CSV', 'Daily_Alert', 'voosh')

In [ ]:
#df_daily_csv

In [ ]:
# weekly extract
#orders,gross,discount,revenue,avg_delivery,avg_prep,new_users,old_users = []
try:

  if Data_Exists_Weekly:
    store_name = df_W_1['Outlet_name'].unique()
    df_daily_csv_W1 =pd.DataFrame(columns=['Week','Total Orders','Gross Revenue $', 'Discount $','Net Revenue $', 'Avg Delivery Time',
                                        'Avg Prep time', 'MFR%', 'New Users', 'Old Users','Total Users'], index=[store_name])
    df_daily_csv_W2 =pd.DataFrame(columns=['Week','Total Orders','Gross Revenue $', 'Discount $','Net Revenue $', 'Avg Delivery Time',
                                        'Avg Prep time', 'MFR%', 'New Users', 'Old Users','Total Users'], index=[store_name])

    for i in store_name:
      df = df_W_1[df_W_1['Outlet_name']==i]
      df_daily_csv_W1.loc[i,'Week'] = week_1
      df_daily_csv_W1.loc[i,'Total Orders'] = df['Order_id'].count()
      df_daily_csv_W1.loc[i,'Net Revenue $'] = df['Revenue'].sum()
      df_daily_csv_W1.loc[i,'Discount $'] = df['Total_discount'].sum()
      df_daily_csv_W1.loc[i,'Gross Revenue $'] = float(df['Revenue'].sum() + df['Total_discount'].sum())

      df_daily_csv_W1.loc[i,'Avg Delivery Time'] = round(df[df['makeline_Delivery_final']<=100]['makeline_Delivery_final'].mean(),2)
      df_daily_csv_W1.loc[i,'Avg Prep time'] = round(df[df['Prep_Time']!=0]['Prep_Time'].mean(),2)
      if (len(df) and len(df[~df['MFR_Marked']]) > 0):
        df_daily_csv_W1.loc[i,'MFR%'] = round((len(df[~df['MFR_Marked']])/len(df))*100,2)
      else:
        df_daily_csv_W1.loc[i,'MFR%'] = 0
      df_daily_csv_W1.loc[i,'New Users'] = len(set(df['Customer_mobile'])) - len(set(df_merged[df_merged['Week']!=week_1]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
      df_daily_csv_W1.loc[i,'Old Users'] = len(set(df_merged[df_merged['Week']!=week_1]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
      df_daily_csv_W1.loc[i,'Total Users'] = len(set(df['Customer_mobile']))

    #  df_daily_csv_T1.loc[i,'New Users'] = len(set(df['Customer_mobile'])) - len(set(df_merged[df_merged['Order_date']!=T_1]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
    #  df_daily_csv_T1.loc[i,'Old Users'] = len(set(df_merged[df_merged['Order_date']!=T_1]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
    #  df_daily_csv_T1.loc[i,'Total Users'] = len(set(df['Customer_mobile']))

    for i in store_name:
      df = df_W_2[df_W_2['Outlet_name']==i]
      df_daily_csv_W2.loc[i,'Week'] = week_2
      df_daily_csv_W2.loc[i,'Total Orders'] = df['Order_id'].count()
      df_daily_csv_W2.loc[i,'Net Revenue $'] = df['Revenue'].sum()
      df_daily_csv_W2.loc[i,'Discount $'] = df['Total_discount'].sum()
      df_daily_csv_W2.loc[i,'Gross Revenue $'] = float(df['Revenue'].sum() + df['Total_discount'].sum())

      df_daily_csv_W2.loc[i,'Avg Delivery Time'] = round(df[df['makeline_Delivery_final']<=100]['makeline_Delivery_final'].mean(),2)
      df_daily_csv_W2.loc[i,'Avg Prep time'] = round(df[df['Prep_Time']!=0]['Prep_Time'].mean(),2)
      if (len(df) and len(df[~df['MFR_Marked']]) > 0):
        df_daily_csv_W2.loc[i,'MFR%'] = round((len(df[~df['MFR_Marked']])/len(df))*100,2)
      else:
        df_daily_csv_W2.loc[i,'MFR%'] = 0
      
      df_daily_csv_W2.loc[i,'Total Users'] = len(set(df['Customer_mobile']))
    
      df_daily_csv_W2.loc[i,'New Users'] = len(set(df['Customer_mobile'])) - len(set(df_merged[(df_merged['Week']!=week_1) & (df_merged['Week']!=week_2)]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
      df_daily_csv_W2.loc[i,'Old Users'] = len(set(df_merged[(df_merged['Week']!=week_1) & (df_merged['Week']!=week_2)]['Customer_mobile']).intersection(set(df['Customer_mobile'])))

    #  daily_alert.loc['New Users','(T-2)'] = len(set(df_merged[df_merged['Order_date']!=T_1]['Customer_mobile']).intersection(set(df_T_1['Customer_mobile'])))
    #  daily_alert.loc['Old Users','(T-2)'] = len(set(df_T_2['Customer_mobile']).difference(set(df_merged[(df_merged['Order_date']!=T_1)&(df_merged['Order_date']!=T_2)]['Customer_mobile'])))

    df_daily_csv_w = pd.concat([df_daily_csv_W1,df_daily_csv_W2])
    df_daily_csv_w.reset_index (inplace=True)
    df_daily_csv_w.rename(columns = {'level_0':'Store Name'}, inplace = True)
    df_daily_csv_w=df_daily_csv_w.sort_values(by=['Store Name'])

except Exception as e:
  errors(e, 'Weekly_CSV', 'Daily_Alert', 'voosh')

In [ ]:
#df_daily_csv_w

In [ ]:
# monthly extract
#orders,gross,discount,revenue,avg_delivery,avg_prep,new_users,old_users = []
try:

  if Data_Exists_Monthly:

    store_name = df_M_1['Outlet_name'].unique()
    df_daily_csv_M1 =pd.DataFrame(columns=['Month','Total Orders','Gross Revenue $', 'Discount $','Net Revenue $', 'Avg Delivery Time',
                                        'Avg Prep time', 'MFR%', 'New Users', 'Old Users'], index=[store_name])
    df_daily_csv_M2 =pd.DataFrame(columns=['Month','Total Orders','Gross Revenue $', 'Discount $','Net Revenue $', 'Avg Delivery Time',
                                        'Avg Prep time', 'MFR%', 'New Users', 'Old Users'], index=[store_name])

    for i in store_name:
      df = df_M_1[df_M_1['Outlet_name']==i]
      df_daily_csv_M1.loc[i,'Month'] = month_1
      df_daily_csv_M1.loc[i,'Total Orders'] = df['Order_id'].count()
      df_daily_csv_M1.loc[i,'Net Revenue $'] = df['Revenue'].sum()
      df_daily_csv_M1.loc[i,'Discount $'] = df['Total_discount'].sum()
      df_daily_csv_M1.loc[i,'Gross Revenue $'] = float(df['Revenue'].sum() + df['Total_discount'].sum())

      df_daily_csv_M1.loc[i,'Avg Delivery Time'] = round(df[df['makeline_Delivery_final']<=100]['makeline_Delivery_final'].mean(),2)
      df_daily_csv_M1.loc[i,'Avg Prep time'] = round(df[df['Prep_Time']!=0]['Prep_Time'].mean(),2)
      if (len(df) and len(df[~df['MFR_Marked']]) > 0):
        df_daily_csv_M1.loc[i,'MFR%'] = round((len(df[~df['MFR_Marked']])/len(df))*100,2)
      else:
        df_daily_csv_W1.loc[i,'MFR%'] = 0
      df_daily_csv_M1.loc[i,'New Users'] = len(set(df['Customer_mobile'])) - len(set(df_merged[df_merged['Month']!=month_1]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
      df_daily_csv_M1.loc[i,'Old Users'] = len(set(df_merged[df_merged['Month']!=month_1]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
      df_daily_csv_M1.loc[i,'Total Users'] = len(set(df['Customer_mobile']))
      
    #  df_daily_csv_M1.loc[i,'New Users'] = len(set(df_merged[df_merged['Order_date']!=T_1]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
    #  df_daily_csv_M1.loc[i,'Old Users'] = len(set(df['Customer_mobile']).difference(set(df_merged[(df_merged['Order_date']!=T_1)]['Customer_mobile'])))

    for i in store_name:
      df = df_M_2[df_M_2['Outlet_name']==i]
      df_daily_csv_M2.loc[i,'Month'] = month_2
      df_daily_csv_M2.loc[i,'Total Orders'] = df['Order_id'].count()
      df_daily_csv_M2.loc[i,'Net Revenue $'] = df['Revenue'].sum()
      df_daily_csv_M2.loc[i,'Discount $'] = df['Total_discount'].sum()
      df_daily_csv_M2.loc[i,'Gross Revenue $'] = float(df['Revenue'].sum() + df['Total_discount'].sum())

      df_daily_csv_M2.loc[i,'Avg Delivery Time'] = round(df[df['makeline_Delivery_final']<=100]['makeline_Delivery_final'].mean(),2)
      df_daily_csv_M2.loc[i,'Avg Prep time'] = round(df[df['Prep_Time']!=0]['Prep_Time'].mean(),2)
      if (len(df) and len(df[~df['MFR_Marked']]) > 0):
        df_daily_csv_M2.loc[i,'MFR%'] = round((len(df[~df['MFR_Marked']])/len(df))*100,2)
      else:
        df_daily_csv_M2.loc[i,'MFR%'] = 0
      df_daily_csv_M2.loc[i,'Total Users'] = len(set(df['Customer_mobile']))
    
      df_daily_csv_M2.loc[i,'New Users'] = len(set(df['Customer_mobile'])) - len(set(df_merged[(df_merged['Month']!=month_1) & (df_merged['Month']!=month_2)]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
      df_daily_csv_M2.loc[i,'Old Users'] = len(set(df_merged[(df_merged['Month']!=month_1) & (df_merged['Month']!=month_2)]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
      
    #  df_daily_csv_M2.loc[i,'New Users'] = len(set(df_merged[df_merged['Order_date']!=T_1]['Customer_mobile']).intersection(set(df['Customer_mobile'])))
    #  df_daily_csv_M2.loc[i,'Old Users'] = len(set(df['Customer_mobile']).difference(set(df_merged[(df_merged['Order_date']!=T_1)&(df_merged['Order_date']!=T_2)]['Customer_mobile'])))

    #  daily_alert.loc['New Users','(T-2)'] = len(set(df_merged[df_merged['Order_date']!=T_1]['Customer_mobile']).intersection(set(df_T_1['Customer_mobile'])))
    #  daily_alert.loc['Old Users','(T-2)'] = len(set(df_T_2['Customer_mobile']).difference(set(df_merged[(df_merged['Order_date']!=T_1)&(df_merged['Order_date']!=T_2)]['Customer_mobile'])))

    df_daily_csv_m = pd.concat([df_daily_csv_M1,df_daily_csv_M2])
    df_daily_csv_m.reset_index (inplace=True)
    df_daily_csv_m.rename(columns = {'level_0':'Store Name'}, inplace = True)
    df_daily_csv_m=df_daily_csv_m.sort_values(by=['Store Name'])


except Exception as e:
  errors(e, 'Monthly_CSV', 'Daily_Alert', 'voosh')

In [ ]:
#df_daily_csv_m

In [ ]:
# Final Write into excel
Created_excel = True
try:
  writer = ExcelWriter('Voosh_Alert.xlsx')
  if Data_Exists_Daily:
    df_daily_csv.to_excel(writer,'Daily by Store',index=True)
  if Data_Exists_Weekly:
    df_daily_csv_w.to_excel(writer,'Weekly by Store',index=True)
  if Data_Exists_Monthly:
    df_daily_csv_m.to_excel(writer,'Monthly by Store',index=True)
  writer.save()

except Exception as e:
  errors(e, 'Write_CSV', 'Daily_Alert', 'voosh')
  Created_excel = False


## FORMAT AND SEND EMAIL ALERT

In [ ]:
#Data_Exists_Daily = False #test

In [ ]:
#from pandas.core.dtypes.base import E
scope_1 = ['https://mail.google.com/']
def gmail_authenticate():
  creds = None
  # the file token.pickle stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first time
  if os.path.exists("/content/token.pickle"):
      with open("/content/token.pickle", "rb") as token:
          creds = pickle.load(token)
  # if there are no (valid) credentials availablle, let the user log in.
  if not creds or not creds.valid:
      if creds and creds.expired and creds.refresh_token:
          creds.refresh(Request())
      else:
          flow = InstalledAppFlow.from_client_secrets_file(
              '/content/gmail_creds.json', scope_1)
          creds = flow.run_local_server(port=0)
      # save the credentials for the next run
      with open("/content/token.pickle", "wb") as token:
          pickle.dump(creds, token)
  return build('gmail', 'v1', credentials=creds)
try:

  if Data_Exists_Daily: # send mail alert
    service = gmail_authenticate()
    html = """<html>
        <head></head>
        <body>
        <head>Hello Team,</head>
        <br>
        
        This is an Overview and Comparison of performace for Yesterday (T-1) Vs Day Before Yesterday (T-2). Similary for Week & Month.
        The overall table consists of various metrics whose comparisons are displayed below.
        Also attached is the storewise breakdown of these metrics for the relevant periods.

        <p>Report Generated Date: {}</p>
        
        
        {}
        <p> </p>
        <p> </p>
        
        
        <p>Thanks & regards,</p>
        <p>Voosh DS Team</p>
        </body>
        <html>""".format(date.today().strftime("%Y-%m-%d"), daily_alert.to_html(index=True),
                      )  # changes

    mimeMessage = MIMEMultipart()

    # to_reciepents = ['sachin@voosh.in','jagruthi@voosh.in','sumit@voosh.in']
    # cc_reciepents = ['kshitiz@voosh.in', 'akshat@voosh.in', 'gaurav@voosh.in', 'priyam@voosh.in','bilal@voosh.in','Radhika@voosh.in','srishti@voosh.in','sumit.c@voosh.in', "zuber@voosh.in"]
    # to_reciepents = ['debidutta@voosh.in', 'iyyappan@voosh.in']
    to_reciepents = ['pallavi@voosh.in']
    cc_reciepents = ['pallavi@voosh.in']
    #     cc_reciepents = [ 'iyyappan@voosh.in', 'roopansh@voosh.in', 'pranit@voosh.in', 'jakir@voosh.in', 'pallavi@voosh.in']
    mimeMessage['to'] = ", ".join(to_reciepents)
    mimeMessage['cc'] = ", ".join(cc_reciepents)

    # send today's date as str
    mimeMessage['subject'] = "Daily email Alert for Voosh -  %s" % (T_1)
    mimeMessage.attach(MIMEText(html, 'html'))
    if Created_excel:
      part = MIMEBase('application', "octet-stream")
      part.set_payload(open('Voosh_Alert.xlsx', "rb").read())
      encoders.encode_base64(part)
      part.add_header('Content-Disposition',
      'attachment; filename="Voosh_Alert.xlsx"')
      mimeMessage.attach(part)
    else:
      print('Excel not attached')
    raw_string = base64.urlsafe_b64encode(mimeMessage.as_bytes()).decode()
    message = service.users().messages().send(
    userId='me', body={'raw': raw_string}).execute()
    print(message)

  else:    #data not created daily - if Data_Exists_Daily: # send mail alert
    service = gmail_authenticate()
    html = """<html>
        <head></head>
        <body>
        <head>Hello Team,</head>
        <br>
        
        Data was not found for T-1 / T-2 days and hence alert was not created. Please re-run after data upload
        to send the alert. 

        <p>Report Not Generated for : {}</p>
        
        
        <p> </p>
        <p> </p>
        
        
        <p>Thanks & regards,</p>
        <p>Voosh DS Team</p>
        </body>
        <html>""".format(date.today().strftime("%Y-%m-%d"))
                        
                        

    mimeMessage = MIMEMultipart()

    # to_reciepents = ['sachin@voosh.in','jagruthi@voosh.in','sumit@voosh.in']
    # cc_reciepents = ['kshitiz@voosh.in', 'akshat@voosh.in', 'gaurav@voosh.in', 'priyam@voosh.in','bilal@voosh.in','Radhika@voosh.in','srishti@voosh.in','sumit.c@voosh.in', "zuber@voosh.in"]
    # to_reciepents = ['debidutta@voosh.in', 'iyyappan@voosh.in']
  #  to_reciepents = ['pallavi@voosh.in','pranit@voosh.in','jakir@voosh.in','iyyappan@voosh.in']
    to_reciepents = ['pallavi@voosh.in']
    cc_reciepents = ['pallavi@voosh.in']
    #     cc_reciepents = [ 'iyyappan@voosh.in', 'roopansh@voosh.in', 'pranit@voosh.in', 'jakir@voosh.in', 'pallavi@voosh.in']
    mimeMessage['to'] = ", ".join(to_reciepents)
    mimeMessage['cc'] = ", ".join(cc_reciepents)

    # send today's date as str
    
    mimeMessage['subject'] = "Daily email Alert Not created for  -  %s" % (T_1)
    mimeMessage.attach(MIMEText(html, 'html'))
    part = MIMEBase('application', "octet-stream")
    raw_string = base64.urlsafe_b64encode(mimeMessage.as_bytes()).decode()
    message = service.users().messages().send(
    userId='me', body={'raw': raw_string}).execute()
    print(message)


except Exception as e:
  errors(e, 'Send_mail', 'Daily_Alert', 'voosh')

{'id': '180d77ebd073dddf', 'threadId': '180d77ebd073dddf', 'labelIds': ['SENT']}


In [ ]:
#daily_alert